<a href="https://colab.research.google.com/gist/sadov/8d8954ee0635c491e9fcae25e27e1620/dateno_llm_wf2-dateno_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Второй этап workflow -- Dateno Search LLM-агент

LLM-агент Dateno Search преобразует запросы произвольной формы в поисковые запросы к базе датасетов через Dateno API.

## Подготовка

Первым делом прописываем токен для доступа к [Hugging Face Spaces Dateno](https://huggingface.co/datenoio/spaces):

In [1]:
HF_TOKEN='YOUR DATENO HF SPACE TOKEN'

Устанавливаем необходимые пакеты:

In [3]:
!pip install git+https://github.com/datenoio/datenollm

  Cloning https://github.com/datenoio/datenollm to /tmp/pip-req-build-9b71u6cc
  Running command git clone --filter=blob:none --quiet https://github.com/datenoio/datenollm /tmp/pip-req-build-9b71u6cc
  Resolved https://github.com/datenoio/datenollm to commit be85bb7dc5af956df40e91c5806a1aeac8f67055
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Импортируем то что понадобится:

In [5]:
import json

from datenollm.client import DatenoClient
from datenollm.jupiter_nb import (
    ChatWidget, DatenoSearchChatWidget, DatenoSearchQuerySelector,
    ask_llm, display_table, dateno2df, create_dateno_search_selector,
    ask_llm_and_create_selector, get_full_path,
    QueriesSelector, QuerySelector,
    )

Стартуем клиент для доступа к HF Space. **М.б. придется позапускать несколько раз если будут ошибки** -- HF Space засыпает если долго не использовался и запускается опять при обращении к нему. При таком раскладе -- заводится пинка с третьего.

In [6]:
client = DatenoClient('datenoio/dateno-search', hf_token=HF_TOKEN)

Loaded as API: https://datenoio-dateno-search.hf.space ✔


Проверим есть ли у нас доступ Google Disk'у. Если его нет, то последует запрос на выдачу разрешений для доступа к Google Account'у.

Google запросит очень много разрешений. Однако, если выставить только разрешение на доступ к файлам и каталогам, то подключения к Google Disk не происходит.

По умолчанию для сохранения в файлах истории и контекста мы используем только каталог /content/drive/MyDrive/colab_data/dateno/, ну или каталог указанный в переменной среды окружения `DRIVE_PATH`, ничего больше. См. https://github.com/datenoio/datenollm/blob/main/datenollm/file_utils.py

Если условие предоставления доступа к данным на персональном Google Disk'е является критическим, можно порекомендовать запускать данный Google Collab notebook на другом аккаунте Google, где важных данных нет.

In [7]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


После предоставления доступа проверяем есть ли каталог /content/drive/MyDrive/colab_data/dateno/, если нет -- он будет создан.

In [8]:
get_full_path('')

'/content/drive/MyDrive/colab_data/dateno/'

##Передача запросов составленных LLM-агентом Query Assistant в Dateno Search

Считываем запросы сгенерированные Query Assitant'ом на предыдущем шаге:

In [9]:
with open(get_full_path('workflow_step1.json'), 'r') as f:
    step1 = json.load(f)

In [10]:
# @title Определяем функцию-обработчик.
def dateno_search(selected_query):
  print(f'Query for Dateno search LLM-agent: "{selected_query[0]['query']}"')
  selector, response, history, error = ask_llm_and_create_selector(client, selected_query[0]['query'])
  if error:
    raise ValueError(f"Error: {error}")
  else:
    selector.display()

In [11]:
# @title Выбираем запрос и отправляем его в Dateno Search:
selector_widget = QuerySelector(queries_data=step1, execute_func=dateno_search)
selector_widget.display()

Query for Dateno search LLM-agent: "total trade turnover data between Armenia and Cyprus from 2023-08-19 to 2025-08-19"
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'


🔍 Executing search: "total trade turnover data from 2023-08-19 to 2025-08-19" with filters: source.countries.name=Armenia, source.countries.name=Cyprus

📊 Search Results for query: "total trade turnover data from 2023-08-19 to 2025-08-19" with filters: source.countries.name=Armenia, source.countries.name=Cyprus
   Filters: source.countries.name=Armenia, source.countries.name=Cyprus
   ❌ No results found
------------------------------
🔍 Executing search: "total trade turnover data between Armenia and Cyprus from 2023-08-19 to 2025-08-19"

📊 Search Results for query: "total trade turnover data between Armenia and Cyprus from 2023-08-19 to 2025-08-19"
   Records found: 500


------------------------------


# Запрос к LLM

In [12]:
# @title Задайте следующие вопросы Dateno Search:
#context_file = "context,json" # @param {"type":"string","placeholder":"Enter History's file name"}
context_file = None
#history_file = "history01.json" # @param {"type":"string","placeholder":"Enter History's file name"}
history_file = None
query = "" # @param {"type":"string","placeholder":"Your query"}

if not query:
  raise ValueError('Query is empty')

selector, response, history, error = ask_llm_and_create_selector(client, query)
#query_text, result_json, history, error = ask_llm(client, query)
if error:
    raise ValueError(f"Error: {error}")
else:
    selector.display()
    #rating_widget = ChatWidget(history_file)
    #display(rating_widget.display())

#import json
#response = json.loads(result_json)
#selector = create_dateno_search_selector(client, response['queries'])
#selector.display()


ValueError: Query is empty